<a href="https://colab.research.google.com/github/shabalin13/code-search/blob/code-cleaning-delivery3/delivery3/PML%26DL_delivery2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Delivery 1

In [ ]:
!pip install datasets --quiet
from datasets import load_dataset


dataset = load_dataset("code_x_glue_ct_code_to_text", 'python')

     |████████████████████████████████| 441 kB 36.3 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
     |████████████████████████████████| 115 kB 67.8 MB/s 
     |████████████████████████████████| 163 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 68.7 MB/s 
     |████████████████████████████████| 115 kB 37.0 MB/s 


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

Dataset code_x_glue_ct_code_to_text downloaded and prepared to /root/.cache/huggingface/datasets/code_x_glue_ct_code_to_text/python/0.0.0/f8b7e9d51f609a87e7ec7c7431706d4ee0b402e3398560410313d4acc67060a0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from pprint import pprint


row = dataset['train'][1]
pprint(row.keys())
print(row['code'])
print(row['docstring'])

dict_keys(['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'])
def setdocument(self, doc):
        """Associate a document with this element.

        Arguments:
            doc (:class:`Document`): A document

        Each element must be associated with a FoLiA document.
        """
        assert isinstance(doc, Document)

        if not self.doc:
            self.doc = doc
            if self.id:
                if self.id in doc:
                    raise DuplicateIDError(self.id)
                else:
                    self.doc.index[id] = self

        for e in self: #recursive for all children
            if isinstance(e,AbstractElement): e.setdocument(doc)
Associate a document with this element.

        Arguments:
            doc (:class:`Document`): A document

        Each element must be associated with a FoLiA document.


In [ ]:
!pip install transformers --quiet


from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

model = AutoModel.from_pretrained("microsoft/codebert-base")

     |████████████████████████████████| 5.3 MB 35.1 MB/s 
     |████████████████████████████████| 7.6 MB 15.7 MB/s 


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
print(tokenizer.cls_token, tokenizer.sep_token, tokenizer.eos_token)
print(tokenizer.all_special_tokens_extended)

<s> </s> </s>
['<s>', '</s>', '<unk>', '<pad>', AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)]


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


def get_emb(code='', nl_text=''):
    nl_tokens=tokenizer.tokenize(nl_text)
    code_tokens=tokenizer.tokenize(code)
    tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
    tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
    t = torch.tensor(tokens_ids).expand(1, -1)
    t = t[None:]
    context_embeddings, embedding = model(t)[0:2]
    return embedding


code1 = "def max(a,b): if a>b: return a else return b"
code2 = "def max(b,c): if b>c: return b else return c"
code3 = "def min(a,b): if a<b: return a else return b"
code4 = "def sub(a,b): return a-b"
code5 = 'hadoop123h'


def measure_distance(emb_func, doc1='', code1='', doc2='', code2=''):
  print('Distance between:')
  print(doc1, code1)
  print(doc2, code2)
  print('is: ' + str(torch.cdist(emb_func(nl_text=doc1, code=code1), emb_func(nl_text=doc2, code=code2))))
  print()
  

measure_distance(code1=code1, code2=code2, emb_func=get_emb)
measure_distance(code1=code1, code2=code3, emb_func=get_emb)
measure_distance(code1=code1, code2=code4, emb_func=get_emb)
measure_distance(code1=code2, code2=code4, emb_func=get_emb)

measure_distance(code1=code1, code2=code5, emb_func=get_emb)

nl_text = 'return maximum value'

measure_distance(code1=code1, doc2=nl_text, emb_func=get_emb)
measure_distance(code1=code2, doc2=nl_text, emb_func=get_emb)
measure_distance(code1=code3, doc2=nl_text, emb_func=get_emb)
measure_distance(code1=code4, doc2=nl_text, emb_func=get_emb)
measure_distance(code1=code5, doc2=nl_text, emb_func=get_emb)

Distance between:
 def max(a,b): if a>b: return a else return b
 def max(b,c): if b>c: return b else return c
is: tensor([[0.1588]], grad_fn=<CdistBackward0>)

Distance between:
 def max(a,b): if a>b: return a else return b
 def min(a,b): if a<b: return a else return b
is: tensor([[0.2051]], grad_fn=<CdistBackward0>)

Distance between:
 def max(a,b): if a>b: return a else return b
 def sub(a,b): return a-b
is: tensor([[0.6079]], grad_fn=<CdistBackward0>)

Distance between:
 def max(b,c): if b>c: return b else return c
 def sub(a,b): return a-b
is: tensor([[0.6281]], grad_fn=<CdistBackward0>)

Distance between:
 def max(a,b): if a>b: return a else return b
 hadoop123h
is: tensor([[1.5200]], grad_fn=<CdistBackward0>)

Distance between:
 def max(a,b): if a>b: return a else return b
return maximum value 
is: tensor([[1.4408]], grad_fn=<CdistBackward0>)

Distance between:
 def max(b,c): if b>c: return b else return c
return maximum value 
is: tensor([[1.4342]], grad_fn=<CdistBackward0>)

Di

#Delivery 2

##Implementing search

In [1]:
!pip install transformers --quiet


from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

model = AutoModel.from_pretrained("microsoft/codebert-base")

     |████████████████████████████████| 5.5 MB 31.0 MB/s 
     |████████████████████████████████| 163 kB 71.2 MB/s 
     |████████████████████████████████| 7.6 MB 56.5 MB/s 


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [3]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
!pip install datasets --quiet
from datasets import load_dataset


dataset = load_dataset("code_x_glue_ct_code_to_text", 'python')

     |████████████████████████████████| 441 kB 38.0 MB/s 
     |████████████████████████████████| 212 kB 75.3 MB/s 
     |████████████████████████████████| 95 kB 5.5 MB/s 
     |████████████████████████████████| 115 kB 67.9 MB/s 
     |████████████████████████████████| 127 kB 68.6 MB/s 
     |████████████████████████████████| 115 kB 69.8 MB/s 


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

Dataset code_x_glue_ct_code_to_text downloaded and prepared to /root/.cache/huggingface/datasets/code_x_glue_ct_code_to_text/python/0.0.0/f8b7e9d51f609a87e7ec7c7431706d4ee0b402e3398560410313d4acc67060a0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
train, validatation, test = dataset['train'], dataset['validation'], dataset['test']

In [ ]:
from torch.utils.data import DataLoader
from enum import Enum, auto

# def format_str(string):
#     for char in ['\r\n', '\r', '\n']:
#         string = string.replace(char, ' ')
#     return string
  
class SeqType(Enum):
  CODE = auto()
  DOC = auto()


class TokenizeCollator(object):
    def __init__(self, tokenizer, seq_type):
        self.tokenizer = tokenizer
        self.seq_type = seq_type

    def __call__(self, batch):
        return self.create_one_batch(batch)

    def create_one_batch(self, batch):
      tokens_batch = list(map(lambda item: self.get_formatted_input(item), batch))
      batch_encoding = self.tokenizer(tokens_batch, padding=True, return_tensors='pt', return_token_type_ids=True, truncation=True)
      tokens_ids = batch_encoding.input_ids.to(device)
      token_type_ids = batch_encoding.token_type_ids.to(device)
      attention_mask = batch_encoding.attention_mask.to(device)
      return tokens_ids, token_type_ids, attention_mask

    def get_formatted_input(self, item):
      if self.seq_type == SeqType.CODE:
        return self.get_formatted_input_for_code(item)
      elif self.seq_type == SeqType.DOC:
        return self.get_fromatted_input_for_doc(item)

    def get_formatted_input_for_code(self, item):
      doc_tokens = ' '.join(item['docstring_tokens'])
      code_tokens = ' '.join(item['code_tokens'])
      formatted_input = self.tokenizer.cls_token + doc_tokens + self.tokenizer.sep_token+code_tokens + self.tokenizer.sep_token
      return formatted_input

    def get_formatted_input_for_doc(self, item):
      doc_tokens = ' '.join(item['docstring_tokens'])
      code_tokens = ''
      formatted_input = self.tokenizer.cls_token + doc_tokens + self.tokenizer.sep_token+code_tokens + self.tokenizer.sep_token
      return formatted_input 


code_tokenize_collate_fn = TokenizeCollator(tokenizer, SeqType.CODE)
doc_tokenize_collate_fn = TokenizeCollator(tokenizer, SeqType.DOC)

BATCH_SIZE = 256
test_code_tokens_ids = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=code_tokenize_collate_fn, num_workers=0)
test_doc_tokens_ids = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=doc_tokenize_collate_fn, num_workers=0)

# for idx, batch in enumerate(test_tokens_ids):
#   # print(batch.shape)
#   print(batch)
#   if idx >= 0:
#     break

In [ ]:
from tqdm import tqdm

# torch.cuda.empty_cache()
model.to(device)


def print_gpu_memory_usage(idx=''):
  print(idx)
  print('Memory Usage:')
  print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
  print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
  print()


test_code_embeddings_list = []
for batch in tqdm(test_code_tokens_ids):
  tokens_ids, token_type_ids, attention_mask = batch
  with torch.no_grad():
    embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
    test_code_embeddings_list.append(embs)

test_doc_embeddings_list = []
for batch in tqdm(test_doc_tokens_ids):
  tokens_ids, token_type_ids, attention_mask = batch
  with torch.no_grad():
    embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
    test_doc_embeddings_list.append(embs)

100%|██████████| 59/59 [00:54<00:00,  1.08it/s]


In [ ]:
test_code_embeddings_concattenated = torch.cat(test_code_embeddings_list, dim=0)
test_doc_embeddings_concattenated = torch.cat(test_doc_embeddings_list, dim=0)

In [ ]:
%cd /content/drive/MyDrive/PML&DL/Project
torch.save(test_code_embeddings_concattenated, 'test_code_embeddings_concattenated.pt')
torch.save(test_doc_embeddings_concattenated, 'test_doc_embeddings_concattenated.pt')

/content/drive/MyDrive/PML&DL/Project


In [ ]:
%cd /content/drive/MyDrive/PML&DL/Project
test_code_embeddings_concattenated = torch.load('test_code_embeddings_concattenated.pt', map_location=torch.device('cpu'))
test_doc_embeddings_concattenated = torch.load('test_doc_embeddings_concattenated.pt', map_location=torch.device('cpu'))

/content/drive/MyDrive/PML&DL/Project


In [ ]:
!apt install libomp-dev
if device == 'cuda':
  !pip install faiss-gpu -q
else:
  !pip install faiss -q

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 27 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 2s (155 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Se

In [ ]:
import numpy as np
import faiss

class FaissKNeighbors:
    def __init__(self, is_cuda):
        self.index = None
        self.is_cuda = is_cuda

    def fit(self, X):
        self.index = faiss.IndexFlatL2(X.shape[1])
        if self.is_cuda:
          res = faiss.StandardGpuResources()
          self.index = faiss.index_cpu_to_gpu(res, 0, self.index)
        if type(X) == torch.Tensor:
          X = X.numpy()
        self.index.add(X)

    def predict(self, X, k):
        if type(X) == torch.Tensor:
          X = X.numpy()
        distances, indices = self.index.search(X, k=k)
        return indices

In [ ]:
test_faiss = FaissKNeighbors(is_cuda=device=='cuda')

In [ ]:
k = 1000
mrrs = []
for beg_idx in tqdm(range(0, len(test_code_embeddings_concattenated), k)):
  if beg_idx + k > len(test_code_embeddings_concattenated):
    break
  doc_embs_subset = test_doc_embeddings_concattenated[beg_idx:beg_idx+k]
  code_embs_subset = test_code_embeddings_concattenated[beg_idx:beg_idx+k]
  test_faiss.fit(code_embs_subset)  
  preds = test_faiss.predict(doc_embs_subset, k=k)

  targets = np.repeat(np.expand_dims(range(k), 1), k, axis=1)

  reciprocal_ranks = 1 / (np.argwhere(np.equal(preds, targets))[:,1] + 1)
  mrr_ = np.mean(reciprocal_ranks)
  mrrs.append(mrr_)

 93%|█████████▎| 14/15 [00:03<00:00,  3.84it/s]


In [ ]:
mrr = np.mean(mrrs)
print('Mean Reciprocal rank is: ', mrr)

Mean Reciprocal rank is:  0.00720989313874937
